In [1]:
# making sure they work...

In [1]:
using Mbo

# Introductory case

In [8]:
s = System("g") # use "g" as a ground state
# Set energy for "e" to 1.5 eV, converted to angular PHz
energy!(s, "e", ev2angphz(1.5)) 
# Set the transition dipole moment for g<->e
dipole!(s, "g", "e", 1.0)
# Define a lineshape functions (some common cases are supplied).
ls(t) = 0.002*t + 0.5*(0.002*t)^2
lineshape!(s, "e", "e", ls)
# setup a calculation grid
tg = TimeGrid(0.0:100, 0.0:100, 0.0:100)
# compute. tedious bookkeeping done by the code.
lin = linear(tg, s) # linear response
rr = R2(tg, s) + R3(tg, s) # rephasing
rn = R1(tg, s) + R4(tg, s) # non-rephasing
# go ahead and process to your wishes.
size(lin), eltype(lin), size(rr), eltype(rr)

((101,), Complex{Float64}, (101, 101, 101), Complex{Float64})

# Longer example from `Basics`

In [9]:
s = System("g")
# set energies
energy!(s, "x1", 1.2)
energy!(s, "x2", 1.4)
# read energies:
@assert energy(s, "x2")-energy(s, "x1") == 1.4-1.2
@assert length(states(s)) == 3
# set transition dipole momments.
dipole!(s, "g", "x1", 1.0) # this sets both g->x1 and x1->g
dipole!(s, "g", "x2", 1.5)
# IMPORTANT: if nonzero, the x1->x2 transition will be part of the calculation
dipole!(s, "x1", "x2", 0)
# read transition dipole
@assert dipole(s, "g", "x1") == dipole(s, "x1", "g")
print(states(s))

String["g", "x1", "x2"]

In [10]:
# set lineshape functions
f(t) = 0.1*t
lineshape!(s, "x1", "x1", f) # accept any callable.
lineshape!(s, "x2", "x2", t->g_homo(t, 0.1)) # anonymous function in Julia: x->...
lineshape!(s, "x1", "x2", zero) # uncorrelated.
lineshape!(s, "x2", "x1", zero) # uncorrelated.
# read lineshape function
@assert lineshape(s, "x1", "x1") === f

In [11]:
t1=0:20.0 # 0 to 20 included in steps of 1.0
t2=0:10.0:100.0 # 0 to 100 included in steps of 10.0
t3=0:20.0
tg = TimeGrid(t1, t2, t3)
rtot = R1(tg, s) + R2(tg, s) + R3(tg, s) + R4(tg, s)
size(tg)

(21, 11, 21)

In [13]:
hpaths = collect(hilbert_paths(s, 3)) # third order hilbert paths
@assert length(hpaths) == 4
# compute for a specific path, pick the first.
p = hpaths[1]
r1 = R1(tg, s, p)
print(p)

<g,x1,g,x1,g>

In [14]:
rlin = linear(tg, s) # use the first time argument.
lin_paths = collect(hilbert_paths(s, 1))
lin_resp = [linear(tg, s, p) for p=lin_paths] # individual contributions.
@assert all(rlin .== sum(lin_resp))